In [1]:
import argparse
import pickle
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Model, GPT2Tokenizer
from torch.utils.data import IterableDataset, Dataset, DataLoader
import torch
import numpy as np

In [2]:
class UciGPT:
    """
    This is a, on UCI notation finetuned, GPT-2 Model with the help of gpt-2-simple.
    See file uic_gpt2.py
    """
    config = GPT2Config.from_pretrained("gpt2-medium")
    config.output_hidden_states = True
    model = GPT2Model.from_pretrained("uci_checkpoint/run1/model-18500.index", from_tf=True, config=config)
    tokenizer = GPT2Tokenizer("uci_checkpoint/run1/encoder.json", "uci_checkpoint/run1/vocab.bpe")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    name = "UciGPT"
    notation = "uci"


In [23]:

# create or open probing_dataset

def train_probing_classifier(chess_model, dataset):
    for x, boards in DataLoader(dataset, batch_size=4):
        chess_model.tokenizer.decode(x['input_ids'])
        #pt_outputs = chess_model.model(**games)  # todo check  shape (batch_size, sequence_length, hidden_size)
        #probing_classifier(pt_outputs.last_hidden_state)
        #exit()



In [24]:

class ChessGamesDataset(IterableDataset):  # todo maybe iterabledataset if memory problems
    def __init__(self, filename, notation, tokenizer, max_len):
        self.filename = filename
        self.notation = notation
        self.tokenizer = tokenizer
        self.tokenizer.model_max_length = max_len

    def game_mapper(self, file_iter):
        game = file_iter.split(";")
        board = np.fromstring(game[-1][1:-1], dtype=int, sep=' ')
        # give uci or pgn to the tokenizer
        return self.tokenizer(game[1] if self.notation == "uci" else game[0], return_tensors='pt', padding='max_length'), board

    def __iter__(self):
        # Create an iterator
        file_itr = open(self.filename, encoding='cp1252')
        # Map each element using the line_mapper
        mapped_itr = map(self.game_mapper, file_itr)
        return mapped_itr
        


In [25]:
for model in [UciGPT]:  # ["pretrained_gpt","pgn_gpt", "uci_gpt", "special_gpt"
    chess_models = [UciGPT]
    for model in chess_models:  # ["pretrained_gpt","pgn_gpt", "uci_gpt", "special_gpt"]
        # get the longest sequence from the tokenizer to do padding later. 
        if model.notation == "uci":
            max_length = model.tokenizer(open("data/max_length_uci.txt").readline(), return_length=True)['length']
        else: # == pgn
            max_length = model.tokenizer(open("data/max_length_pgn.txt").readline(), return_length=True)['length'].item()

        # create dataset. 
        dataset = ChessGamesDataset("data/probing_dataset.txt", model.notation, model.tokenizer, max_length)
        # todo split()
        train_probing_classifier(model, dataset)
        #train_probing_classifier(model, dataset, probing_classifier)
        
        # todo save

<ipython-input-24-c40242c95634>:10: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  board = np.fromstring(game[-1][1:-1], dtype=int, sep=' ')


TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'